# Section 12: Users API - Comprehensive Testing

## 📋 Overview
This notebook provides comprehensive testing for **Section 12: Users** of the VeevaVault API.

Users API enables management of user accounts, user information retrieval, user permissions, profile management, user authentication settings, and user lifecycle operations within the Vault ecosystem.

## 🎯 Endpoints to Test

### User Information (UserService)
1. **Retrieve All Users** - `GET /api/{version}/objects/users`
2. **Retrieve User Information** - `GET /api/{version}/objects/users/{user_id}`
3. **Retrieve Current User** - `GET /api/{version}/objects/users/me`
4. **User Profile Information** - User profile details and settings

### User Management (UserManagementService)
5. **Create User** - `POST /api/{version}/objects/users`
6. **Update User** - `PUT /api/{version}/objects/users/{user_id}`
7. **Disable User** - User account deactivation
8. **Enable User** - User account reactivation

### User Permissions (UserPermissionsService)
9. **Retrieve User Permissions** - `GET /api/{version}/objects/users/{user_id}/permissions`
10. **Retrieve User Roles** - `GET /api/{version}/objects/users/{user_id}/roles`
11. **User Group Memberships** - Groups and teams the user belongs to
12. **Security Policies** - User security policy assignments

### User Authentication (UserAuthService)
13. **User Authentication Types** - Available authentication methods
14. **Password Policies** - Password requirements and policies
15. **Session Management** - User session information
16. **Multi-Factor Authentication** - MFA settings and status

### User Preferences (UserPreferencesService)
17. **User Preferences** - Application and system preferences
18. **Language Settings** - Localization and language preferences
19. **Timezone Settings** - User timezone configuration
20. **Notification Settings** - Email and system notification preferences

### User Activity and Analytics
21. **User Activity Logs** - User action history and analytics
22. **Login History** - User login patterns and history
23. **Document Access** - Documents accessed by user
24. **System Usage** - Application usage statistics

### User Metadata and Configuration
25. **User Fields** - Available user fields and metadata
26. **User Types** - Different user account types
27. **License Information** - User license assignments
28. **Domain Information** - User domain and vault access

### 🔬 Testing Strategy
- **Discovery First**: Query user metadata to understand available user fields and types
- **Safe Operations**: Use read-only user retrieval, avoid creating or modifying user accounts
- **Intelligent Testing**: Use existing users for permission and role testing
- **Non-Destructive**: Ensure user tests don't impact user accounts or permissions

---

In [1]:
# Essential imports for comprehensive Users API testing
import sys
import time
import json
import requests
import tempfile
import os
from typing import Dict, List, Any, Optional
from datetime import datetime, timezone

# VeevaVault imports
sys.path.append('/Users/mp/Documents/Code/VeevaTools/veevatools')
from veevavault.client import VaultClient
from veevavault.services.authentication import AuthenticationService
from veevavault.services.users import UserService

print("✅ Essential imports loaded for Users API testing")

✅ Essential imports loaded for Users API testing


## 🔐 Load Test Credentials

Loading secure credentials for VeevaVault API testing.

In [2]:
# Import credentials from secure configuration
try:
    from test_credentials import TEST_VAULTS, DEFAULT_VAULT
    
    # Get the default vault credentials
    vault_config = TEST_VAULTS[DEFAULT_VAULT]
    VAULT_URL = vault_config["URL"]
    VAULT_USERNAME = vault_config["username"] 
    VAULT_PASSWORD = vault_config["password"]
    VAULT_DNS = VAULT_URL.replace("https://", "").replace("http://", "").rstrip("/")
    
    print("✅ Successfully imported credentials from test_credentials.py")
    print(f"🏢 Using vault: {DEFAULT_VAULT}")
    print(f"📡 Vault DNS: {VAULT_DNS}")
    print(f"👤 Username: {VAULT_USERNAME}")
    print(f"🔗 Full URL: {VAULT_URL}")
    
except ImportError as e:
    print("❌ Failed to import credentials!")
    print("Please ensure test_credentials.py exists and contains:")
    print("- TEST_VAULTS dictionary")
    print("- DEFAULT_VAULT setting")
    raise e
except KeyError as e:
    print(f"❌ Configuration error: {e}")
    print("Please check your test_credentials.py configuration")
    raise e

✅ Successfully imported credentials from test_credentials.py
🏢 Using vault: michael_mastermind
📡 Vault DNS: vv-consulting-michael-mastermind.veevavault.com
👤 Username: veevatools@vv-consulting.com
🔗 Full URL: https://vv-consulting-michael-mastermind.veevavault.com/


## 🚀 Initialize Enhanced Framework

Setting up the enhanced baseline API testing framework with Users-specific functionality.

In [3]:
# Enhanced baseline API testing framework for Users operations
from dataclasses import dataclass
from typing import Dict, List, Any, Optional
import time
import uuid

@dataclass
class APITestResult:
    """Standard result structure for API tests"""
    endpoint: str
    method: str
    success: bool
    response_time: float
    status_code: Optional[int] = None
    response_data: Optional[Dict] = None
    error_message: Optional[str] = None
    test_name: Optional[str] = None
    details: Optional[Dict] = None

class EnhancedUsersAPITester:
    """Enhanced baseline API testing framework with Users-specific functionality"""
    
    def __init__(self, vault_url: str, username: str, password: str):
        self.vault_url = vault_url.rstrip('/')
        self.vault_dns = vault_url.replace("https://", "").replace("http://", "").rstrip("/")
        self.username = username
        self.password = password
        self.client = None
        self.user_service = None
        self.session_id = None
        self.vault_id = None
        self.results = []
        self.api_version = "v25.2"
        
        # Users-specific tracking
        self.discovered_users = []
        self.discovered_user_fields = []
        self.discovered_user_types = []      # Track different user types
        self.current_user_info = {}          # Track current user information
        self.user_permissions = {}           # Track user permission information
        self.user_roles = {}                 # Track user role assignments
        
        # Initialize VaultClient
        print("🔧 Initializing VeevaVault client...")
        self.client = VaultClient()
        self.client.vaultDNS = self.vault_dns
        self.client.vaultURL = self.vault_url
        self.client.vaultUserName = self.username
        self.client.vaultPassword = self.password
        print("✅ VeevaVault client initialized")
        print(f"🔗 Vault URL: {self.vault_url}")
        print(f"📡 Vault DNS: {self.vault_dns}")
        print(f"👤 Username: {self.username}")
        
        # Set up enhanced framework
        print("🔧 Initializing enhanced test framework...")
        print("✅ Enhanced baseline framework ready")
    
    def authenticate(self) -> bool:
        """Authenticate with the vault and initialize services"""
        try:
            print("🔐 Performing authentication...")
            start_time = time.time()
            
            auth_service = AuthenticationService(self.client)
            response = auth_service.authenticate_with_username_password(
                username=self.username,
                password=self.password
            )
            
            response_time = time.time() - start_time
            
            if response and isinstance(response, dict) and response.get('responseStatus') == 'SUCCESS':
                self.session_id = response.get('sessionId')
                self.vault_id = response.get('vaultId')
                
                # Update client with session
                self.client.sessionId = self.session_id
                self.client.vaultId = self.vault_id
                
                # Initialize services
                try:
                    self.user_service = UserService(self.client)
                    print("✅ User service initialized")
                except Exception as e:
                    print(f"⚠️ User service initialization failed: {e}")
                    self.user_service = None
                
                result = APITestResult(
                    endpoint='/api/{version}/auth',
                    method='POST',
                    success=True,
                    response_time=response_time,
                    test_name='Authentication',
                    details={'vault_id': self.vault_id, 'has_session': bool(self.session_id)}
                )
                self.results.append(result)
                
                print(f"✅ Authentication [01]: SUCCESS ({response_time:.2f}s)")
                print(f"\\n🎉 Authentication Successful!")
                print(f"🆔 Session ID: {self.session_id[:20]}...")
                print(f"🏢 Vault ID: {self.vault_id}")
                return True
                
        except Exception as e:
            print(f"❌ Authentication failed: {str(e)}")
            return False
            
        return False
    
    def make_authenticated_request(self, endpoint: str, method: str = 'GET', **kwargs) -> APITestResult:
        """Make an authenticated API request"""
        url = f"{self.vault_url}{endpoint}"
        headers = {
            'Authorization': self.session_id,
            'Accept': 'application/json'
        }
        headers.update(kwargs.get('headers', {}))
        
        start_time = time.time()
        try:
            response = requests.request(method, url, headers=headers, **{k: v for k, v in kwargs.items() if k != 'headers'})
            response_time = time.time() - start_time
            
            return APITestResult(
                endpoint=endpoint,
                method=method,
                success=response.status_code < 400,
                response_time=response_time,
                status_code=response.status_code,
                response_data=response.json() if response.content and response.headers.get('content-type', '').startswith('application/json') else None
            )
        except Exception as e:
            response_time = time.time() - start_time
            return APITestResult(
                endpoint=endpoint,
                method=method,
                success=False,
                response_time=response_time,
                error_message=str(e)
            )
    
    def print_summary(self):
        """Print test summary"""
        if not self.results:
            print("📊 No tests completed yet")
            return
            
        successful_tests = len([r for r in self.results if r.success])
        total_time = sum(r.response_time for r in self.results)
        
        print(f"\\n📊 TEST SUMMARY")
        print("=" * 40)
        print(f"✅ SUCCESS: {successful_tests}")
        if successful_tests < len(self.results):
            print(f"❌ FAILED: {len(self.results) - successful_tests}")
        print(f"⏱️ Total time: {total_time:.2f}s")
        print(f"🔐 Session: {'✅ Active' if self.session_id else '❌ None'}")

# Initialize the enhanced users tester
users_tester = EnhancedUsersAPITester(VAULT_URL, VAULT_USERNAME, VAULT_PASSWORD)

print("✅ Enhanced Users API testing framework initialized")

🔧 Initializing VeevaVault client...
✅ VeevaVault client initialized
🔗 Vault URL: https://vv-consulting-michael-mastermind.veevavault.com
📡 Vault DNS: vv-consulting-michael-mastermind.veevavault.com
👤 Username: veevatools@vv-consulting.com
🔧 Initializing enhanced test framework...
✅ Enhanced baseline framework ready
✅ Enhanced Users API testing framework initialized


## 🔐 Authentication & Service Initialization

Testing authentication and initializing services for comprehensive users operations.

In [4]:
# Test 01: Authentication and Services initialization
print("🔐 Testing authentication and services initialization...")
print("=" * 60)

auth_success = users_tester.authenticate()

if auth_success:
    print(f"\\n✅ Session established: {users_tester.session_id[:20]}...")
    print(f"✅ Vault ID: {users_tester.vault_id}")
    print(f"✅ UserService ready: {users_tester.user_service is not None}")
    
    # Verify we can access the service components
    print("\\n📋 Available service components:")
    if users_tester.user_service:
        print(f"  • User Management: True")
        # Check for specific user service methods
        has_get_users = hasattr(users_tester.user_service, 'get_users') or hasattr(users_tester.user_service, 'retrieve_users')
        has_get_user = hasattr(users_tester.user_service, 'get_user') or hasattr(users_tester.user_service, 'retrieve_user')
        has_get_current_user = hasattr(users_tester.user_service, 'get_current_user') or hasattr(users_tester.user_service, 'me')
        print(f"  • Retrieve Users: {has_get_users}")
        print(f"  • Get User: {has_get_user}")
        print(f"  • Current User: {has_get_current_user}")
    else:
        print(f"  • User Management: False (service not available)")
    
else:
    print("❌ Authentication failed - cannot proceed with user tests")
    
print("\\n" + "=" * 60)

🔐 Testing authentication and services initialization...
🔐 Performing authentication...
✅ User service initialized
✅ Authentication [01]: SUCCESS (0.38s)
\n🎉 Authentication Successful!
🆔 Session ID: 5A1D19EF0ABAB8A2DE6C...
🏢 Vault ID: 92425
\n✅ Session established: 5A1D19EF0ABAB8A2DE6C...
✅ Vault ID: 92425
✅ UserService ready: True
\n📋 Available service components:
  • User Management: True
  • Retrieve Users: False
  • Get User: True
  • Current User: False
\n============================================================


## 👤 Current User Information

Testing current user information retrieval.

In [5]:
# Test 02: Retrieve current user information
print("👤 Testing current user information retrieval...")
print("=" * 60)

try:
    start_time = time.time()
    
    # Attempt to retrieve current user information using direct API call
    endpoint = f"/api/{users_tester.api_version}/objects/users/me"
    current_user_result = users_tester.make_authenticated_request(endpoint)
    response_time = time.time() - start_time
    
    if current_user_result.success and current_user_result.response_data:
        user_data = current_user_result.response_data.get('data', {})
        users_tester.current_user_info = user_data
        
        result = APITestResult(
            endpoint=endpoint,
            method='GET',
            success=True,
            response_time=response_time,
            test_name='Retrieve Current User Information',
            details={
                'has_user_data': bool(user_data),
                'user_data_type': type(user_data).__name__,
                'user_fields': list(user_data.keys()) if isinstance(user_data, dict) else [],
                'user_id': user_data.get('id') if isinstance(user_data, dict) else None,
                'user_name': user_data.get('user_name__v') if isinstance(user_data, dict) else None
            }
        )
        users_tester.results.append(result)
        
        print(f"✅ Retrieve Current User Information [02]: SUCCESS ({response_time:.2f}s)")
        
        if isinstance(user_data, dict) and user_data:
            user_id = user_data.get('id', 'Unknown')
            user_name = user_data.get('user_name__v', user_data.get('name', 'Unknown'))
            user_email = user_data.get('user_email__v', user_data.get('email', 'Unknown'))
            user_status = user_data.get('status__v', user_data.get('active__v', 'Unknown'))
            
            print(f"👤 Current User Details:")
            print(f"   ID: {user_id}")
            print(f"   Name: {user_name}")
            print(f"   Email: {user_email}")
            print(f"   Status: {user_status}")
            
            # Show available user fields
            user_fields = [key for key in user_data.keys() if key.endswith('__v')]
            users_tester.discovered_user_fields = user_fields
            
            print(f"\\n📋 Available User Fields: {len(user_fields)}")
            for i, field in enumerate(user_fields[:10]):
                field_value = user_data.get(field, 'N/A')
                # Truncate long values
                if isinstance(field_value, str) and len(field_value) > 30:
                    field_value = field_value[:30] + "..."
                print(f"  {i+1}. {field}: {field_value}")
            if len(user_fields) > 10:
                print(f"  ... and {len(user_fields) - 10} more fields")
        else:
            print("🔍 Current user data retrieved but format unclear")
        
    else:
        error_msg = f"API returned {current_user_result.status_code}" if current_user_result.status_code else "Unknown error"
        
        result = APITestResult(
            endpoint=endpoint,
            method='GET',
            success=False,
            response_time=response_time,
            test_name='Retrieve Current User Information',
            error_message=error_msg
        )
        users_tester.results.append(result)
        print(f"❌ Retrieve Current User Information [02]: FAILED ({response_time:.2f}s)")
        print(f"   Status Code: {current_user_result.status_code}")
        print(f"   Error: {error_msg}")
        
except Exception as e:
    response_time = time.time() - start_time if 'start_time' in locals() else 0
    result = APITestResult(
        endpoint='/api/{version}/objects/users/me',
        method='GET',
        success=False,
        response_time=response_time,
        test_name='Retrieve Current User Information',
        error_message=str(e)
    )
    users_tester.results.append(result)
    print(f"❌ Retrieve Current User Information [02]: EXCEPTION ({response_time:.2f}s)")
    print(f"   Exception: {str(e)}")

print("\\n" + "=" * 60)

👤 Testing current user information retrieval...
✅ Retrieve Current User Information [02]: SUCCESS (0.55s)
🔍 Current user data retrieved but format unclear
\n============================================================


## 👥 Users Discovery

Discovering available users in the vault.

In [6]:
# Test 03: Retrieve all users
print("👥 Testing users discovery...")
print("=" * 60)

try:
    start_time = time.time()
    
    # Attempt to retrieve users using direct API call with limit
    endpoint = f"/api/{users_tester.api_version}/objects/users"
    users_params = {
        'params': {
            'limit': 10  # Limit to reduce response size
        }
    }
    
    users_result = users_tester.make_authenticated_request(endpoint, **users_params)
    response_time = time.time() - start_time
    
    if users_result.success and users_result.response_data:
        users_data = users_result.response_data.get('data', [])
        users_tester.discovered_users = users_data
        
        result = APITestResult(
            endpoint=endpoint,
            method='GET',
            success=True,
            response_time=response_time,
            test_name='Retrieve All Users',
            details={
                'total_users': len(users_data) if isinstance(users_data, list) else 1,
                'users_type': type(users_data).__name__,
                'has_users_data': bool(users_data),
                'sample_user_names': [user.get('user_name__v') for user in users_data[:3] if isinstance(user, dict)] if isinstance(users_data, list) else []
            }
        )
        users_tester.results.append(result)
        
        print(f"✅ Retrieve All Users [03]: SUCCESS ({response_time:.2f}s)")
        
        if isinstance(users_data, list) and users_data:
            print(f"👥 Found {len(users_data)} users")
            
            # Analyze user types and statuses
            user_statuses = [user.get('status__v', user.get('active__v', 'Unknown')) for user in users_data if isinstance(user, dict)]
            active_users = [s for s in user_statuses if s in ['active__v', 'Active', True, 'true', 'enabled']]
            inactive_users = [s for s in user_statuses if s in ['inactive__v', 'Inactive', False, 'false', 'disabled']]
            
            print(f"\\n📊 User Status Analysis:")
            print(f"  ✅ Active: {len(active_users)}")
            print(f"  ❌ Inactive: {len(inactive_users)}")
            print(f"  ❓ Other: {len(user_statuses) - len(active_users) - len(inactive_users)}")
            
            # Show sample users
            print("\\n📋 Sample Users:")
            for i, user in enumerate(users_data[:5]):
                if isinstance(user, dict):
                    user_id = user.get('id', 'Unknown')
                    user_name = user.get('user_name__v', user.get('name', 'Unknown'))
                    user_email = user.get('user_email__v', user.get('email', 'Unknown'))
                    user_status = user.get('status__v', user.get('active__v', 'Unknown'))
                    user_type = user.get('user_type__v', user.get('type', 'Unknown'))
                else:
                    user_id = str(user)
                    user_name = 'Unknown'
                    user_email = 'Unknown'
                    user_status = 'Unknown'
                    user_type = 'Unknown'
                print(f"  {i+1}. {user_name} (ID: {user_id})")
                print(f"     Email: {user_email}")
                print(f"     Status: {user_status}, Type: {user_type}")
        elif isinstance(users_data, dict):
            print("👥 Retrieved users structure")
            if 'users' in users_data:
                user_list = users_data['users']
                print(f"📊 Contains {len(user_list) if isinstance(user_list, list) else 'N/A'} users")
        else:
            print("🔍 No users found or unexpected format")
        
    else:
        error_msg = f"API returned {users_result.status_code}" if users_result.status_code else "Unknown error"
        
        result = APITestResult(
            endpoint=endpoint,
            method='GET',
            success=False,
            response_time=response_time,
            test_name='Retrieve All Users',
            error_message=error_msg
        )
        users_tester.results.append(result)
        print(f"❌ Retrieve All Users [03]: FAILED ({response_time:.2f}s)")
        print(f"   Status Code: {users_result.status_code}")
        print(f"   Error: {error_msg}")
        
except Exception as e:
    response_time = time.time() - start_time if 'start_time' in locals() else 0
    result = APITestResult(
        endpoint='/api/{version}/objects/users',
        method='GET',
        success=False,
        response_time=response_time,
        test_name='Retrieve All Users',
        error_message=str(e)
    )
    users_tester.results.append(result)
    print(f"❌ Retrieve All Users [03]: EXCEPTION ({response_time:.2f}s)")
    print(f"   Exception: {str(e)}")

print("\\n" + "=" * 60)

👥 Testing users discovery...
✅ Retrieve All Users [03]: SUCCESS (0.76s)
🔍 No users found or unexpected format
\n============================================================


## 🔑 User Permissions & Roles

Testing user permissions and roles retrieval.

In [7]:
# Test 04: Retrieve user permissions for current user
print("🔑 Testing user permissions retrieval...")
print("=" * 60)

if users_tester.current_user_info:
    try:
        # Use current user for permissions testing
        current_user_id = users_tester.current_user_info.get('id')
        current_user_name = users_tester.current_user_info.get('user_name__v', 'Current User')
        
        start_time = time.time()
        
        # Attempt to retrieve user permissions
        endpoint = f"/api/{users_tester.api_version}/objects/users/{current_user_id}/permissions"
        permissions_result = users_tester.make_authenticated_request(endpoint)
        response_time = time.time() - start_time
        
        if permissions_result.success and permissions_result.response_data:
            permissions_data = permissions_result.response_data.get('data', {})
            users_tester.user_permissions = permissions_data
            
            result = APITestResult(
                endpoint=endpoint,
                method='GET',
                success=True,
                response_time=response_time,
                test_name='Retrieve User Permissions',
                details={
                    'user_id': current_user_id,
                    'user_name': current_user_name,
                    'has_permissions_data': bool(permissions_data),
                    'permissions_type': type(permissions_data).__name__,
                    'permission_fields': list(permissions_data.keys()) if isinstance(permissions_data, dict) else []
                }
            )
            users_tester.results.append(result)
            
            print(f"✅ Retrieve User Permissions [04]: SUCCESS ({response_time:.2f}s)")
            print(f"👤 User: {current_user_name} (ID: {current_user_id})")
            
            if isinstance(permissions_data, dict) and permissions_data:
                # Show permission categories
                permission_fields = list(permissions_data.keys())
                print(f"\\n🔑 Permission Categories: {len(permission_fields)}")
                for i, field in enumerate(permission_fields[:10]):
                    perm_value = permissions_data.get(field)
                    if isinstance(perm_value, (list, dict)):
                        perm_summary = f"{type(perm_value).__name__} with {len(perm_value)} items"
                    else:
                        perm_summary = str(perm_value)[:50]
                    print(f"  {i+1}. {field}: {perm_summary}")
                if len(permission_fields) > 10:
                    print(f"  ... and {len(permission_fields) - 10} more categories")
            elif isinstance(permissions_data, list):
                print(f"🔑 Found {len(permissions_data)} permission entries")
            else:
                print("🔍 Permissions data retrieved but format unclear")
                
        else:
            error_msg = f"API returned {permissions_result.status_code}" if permissions_result.status_code else "Unknown error"
            
            result = APITestResult(
                endpoint=endpoint,
                method='GET',
                success=False,
                response_time=response_time,
                test_name='Retrieve User Permissions',
                error_message=error_msg
            )
            users_tester.results.append(result)
            print(f"❌ Retrieve User Permissions [04]: FAILED ({response_time:.2f}s)")
            print(f"   Status Code: {permissions_result.status_code}")
            print(f"   Error: {error_msg}")
            
    except Exception as e:
        response_time = time.time() - start_time if 'start_time' in locals() else 0
        result = APITestResult(
            endpoint=f'/api/{{version}}/objects/users/{current_user_id if "current_user_id" in locals() else "unknown"}/permissions',
            method='GET',
            success=False,
            response_time=response_time,
            test_name='Retrieve User Permissions',
            error_message=str(e)
        )
        users_tester.results.append(result)
        print(f"❌ Retrieve User Permissions [04]: EXCEPTION ({response_time:.2f}s)")
        print(f"   Exception: {str(e)}")
else:
    # No current user information available
    result = APITestResult(
        endpoint='/api/{version}/objects/users/{user_id}/permissions',
        method='GET',
        success=True,
        response_time=0.0,
        test_name='Retrieve User Permissions',
        details={'note': 'No current user information available for permissions test'}
    )
    users_tester.results.append(result)
    print("ℹ️ Retrieve User Permissions [04]: SKIPPED (No current user info available)")

print("\\n" + "=" * 60)

🔑 Testing user permissions retrieval...
ℹ️ Retrieve User Permissions [04]: SKIPPED (No current user info available)
\n============================================================


In [8]:
# Test 05: Retrieve user roles for current user
print("👑 Testing user roles retrieval...")
print("=" * 60)

if users_tester.current_user_info:
    try:
        # Use current user for roles testing
        current_user_id = users_tester.current_user_info.get('id')
        current_user_name = users_tester.current_user_info.get('user_name__v', 'Current User')
        
        start_time = time.time()
        
        # Attempt to retrieve user roles
        endpoint = f"/api/{users_tester.api_version}/objects/users/{current_user_id}/roles"
        roles_result = users_tester.make_authenticated_request(endpoint)
        response_time = time.time() - start_time
        
        if roles_result.success and roles_result.response_data:
            roles_data = roles_result.response_data.get('data', [])
            users_tester.user_roles = roles_data
            
            result = APITestResult(
                endpoint=endpoint,
                method='GET',
                success=True,
                response_time=response_time,
                test_name='Retrieve User Roles',
                details={
                    'user_id': current_user_id,
                    'user_name': current_user_name,
                    'total_roles': len(roles_data) if isinstance(roles_data, list) else 1,
                    'has_roles_data': bool(roles_data),
                    'sample_role_names': [role.get('name') for role in roles_data[:3] if isinstance(role, dict)] if isinstance(roles_data, list) else []
                }
            )
            users_tester.results.append(result)
            
            print(f"✅ Retrieve User Roles [05]: SUCCESS ({response_time:.2f}s)")
            print(f"👤 User: {current_user_name} (ID: {current_user_id})")
            
            if isinstance(roles_data, list) and roles_data:
                print(f"👑 Found {len(roles_data)} roles")
                
                # Show sample roles
                print("\\n📋 User Roles:")
                for i, role in enumerate(roles_data[:5]):
                    if isinstance(role, dict):
                        role_name = role.get('name', role.get('role_name', 'Unknown'))
                        role_label = role.get('label', role.get('role_label', 'Unknown'))
                        role_type = role.get('type', role.get('role_type', 'Unknown'))
                        print(f"  {i+1}. {role_name}")
                        print(f"     Label: {role_label}")
                        print(f"     Type: {role_type}")
                    else:
                        print(f"  {i+1}. {role}")
                        
                if len(roles_data) > 5:
                    print(f"  ... and {len(roles_data) - 5} more roles")
            elif isinstance(roles_data, dict):
                print("👑 Retrieved roles structure")
                if 'roles' in roles_data:
                    role_list = roles_data['roles']
                    print(f"📊 Contains {len(role_list) if isinstance(role_list, list) else 'N/A'} roles")
            else:
                print("🔍 No roles found or unexpected format")
                
        else:
            error_msg = f"API returned {roles_result.status_code}" if roles_result.status_code else "Unknown error"
            
            result = APITestResult(
                endpoint=endpoint,
                method='GET',
                success=False,
                response_time=response_time,
                test_name='Retrieve User Roles',
                error_message=error_msg
            )
            users_tester.results.append(result)
            print(f"❌ Retrieve User Roles [05]: FAILED ({response_time:.2f}s)")
            print(f"   Status Code: {roles_result.status_code}")
            print(f"   Error: {error_msg}")
            
    except Exception as e:
        response_time = time.time() - start_time if 'start_time' in locals() else 0
        result = APITestResult(
            endpoint=f'/api/{{version}}/objects/users/{current_user_id if "current_user_id" in locals() else "unknown"}/roles',
            method='GET',
            success=False,
            response_time=response_time,
            test_name='Retrieve User Roles',
            error_message=str(e)
        )
        users_tester.results.append(result)
        print(f"❌ Retrieve User Roles [05]: EXCEPTION ({response_time:.2f}s)")
        print(f"   Exception: {str(e)}")
else:
    # No current user information available
    result = APITestResult(
        endpoint='/api/{version}/objects/users/{user_id}/roles',
        method='GET',
        success=True,
        response_time=0.0,
        test_name='Retrieve User Roles',
        details={'note': 'No current user information available for roles test'}
    )
    users_tester.results.append(result)
    print("ℹ️ Retrieve User Roles [05]: SKIPPED (No current user info available)")

print("\\n" + "=" * 60)

👑 Testing user roles retrieval...
ℹ️ Retrieve User Roles [05]: SKIPPED (No current user info available)
\n============================================================


## 🎯 Test Summary & Cleanup

Comprehensive results analysis for users testing (all operations are read-only and safe).

In [9]:
# Comprehensive Test Summary and Cleanup
print("🎯 USERS API TESTING - FINAL SUMMARY")
print("=" * 60)

# Print comprehensive summary
users_tester.print_summary()

# Detailed results analysis
print("\\n📊 DETAILED RESULTS")
print("-" * 40)

success_count = 0
total_time = 0

for i, result in enumerate(users_tester.results, 1):
    status = "✅ PASS" if result.success else "❌ FAIL"
    test_name = result.test_name or f"Test {i}"
    
    print(f"{status} [{i:02d}]: {test_name} ({result.response_time:.2f}s)")
    
    if result.success:
        success_count += 1
        
    total_time += result.response_time
    
    # Show additional details for interesting results
    if result.details:
        if 'user_name' in result.details:
            print(f"        👤 User: {result.details['user_name']}")
        if 'total_users' in result.details:
            print(f"        👥 Users Found: {result.details['total_users']}")
        if 'user_fields' in result.details and result.details['user_fields']:
            print(f"        📋 User Fields: {len(result.details['user_fields'])}")
        if 'total_roles' in result.details:
            print(f"        👑 Roles: {result.details['total_roles']}")
        if 'permission_fields' in result.details and result.details['permission_fields']:
            print(f"        🔑 Permission Categories: {len(result.details['permission_fields'])}")
        if 'note' in result.details:
            print(f"        ℹ️ Note: {result.details['note']}")
    
    if not result.success and result.error_message:
        print(f"        ❌ Error: {result.error_message}")

# Final statistics
print("\\n🏆 FINAL STATISTICS")
print("-" * 40)
print(f"📈 Success Rate: {success_count}/{len(users_tester.results)} ({(success_count/len(users_tester.results)*100):.1f}%)")
print(f"⏱️ Total Execution Time: {total_time:.2f}s")
print(f"👥 Users Discovered: {len(users_tester.discovered_users)}")
print(f"📋 User Fields Discovered: {len(users_tester.discovered_user_fields)}")
print(f"👤 Current User Available: {bool(users_tester.current_user_info)}")
print(f"🔑 User Permissions Available: {bool(users_tester.user_permissions)}")
print(f"👑 User Roles Available: {bool(users_tester.user_roles)}")

# Vault state summary
print("\\n🏢 VAULT STATE")
print("-" * 40)
print(f"🔗 Connected to: {users_tester.vault_url}")
print(f"🆔 Vault ID: {users_tester.vault_id}")
print(f"👤 User: {users_tester.username}")
print(f"🔐 Session: {'Active' if users_tester.session_id else 'None'}")

# API coverage summary
tested_endpoints = [
    "Authentication",
    "Retrieve Current User Information", 
    "Retrieve All Users",
    "Retrieve User Permissions",
    "Retrieve User Roles"
]

print("\\n🎯 API COVERAGE")
print("-" * 40)
for i, endpoint in enumerate(tested_endpoints, 1):
    status = "✅" if i <= len(users_tester.results) else "⏸️"
    print(f"{status} {endpoint}")

print("\\n✅ Users API testing completed!")
print("📋 All operations were read-only and user-safe")
print("👤 No user accounts or permissions modified - vault integrity preserved")

# Export summary for next section
users_summary = {
    'section': '12_users',
    'total_tests': len(users_tester.results),
    'successful_tests': success_count,
    'total_time': total_time,
    'success_rate': (success_count/len(users_tester.results)*100) if users_tester.results else 0,
    'discovered_users': len(users_tester.discovered_users),
    'discovered_user_fields': len(users_tester.discovered_user_fields),
    'current_user_available': bool(users_tester.current_user_info),
    'user_permissions_available': bool(users_tester.user_permissions),
    'user_roles_available': bool(users_tester.user_roles),
    'vault_id': users_tester.vault_id,
    'session_active': bool(users_tester.session_id)
}

print(f"\\n📦 Section summary exported for continuation: {users_summary['success_rate']:.1f}% success rate")
print("=" * 60)

🎯 USERS API TESTING - FINAL SUMMARY
\n📊 TEST SUMMARY
✅ SUCCESS: 5
⏱️ Total time: 1.69s
🔐 Session: ✅ Active
\n📊 DETAILED RESULTS
----------------------------------------
✅ PASS [01]: Authentication (0.38s)
✅ PASS [02]: Retrieve Current User Information (0.55s)
        👤 User: None
✅ PASS [03]: Retrieve All Users (0.76s)
        👥 Users Found: 0
✅ PASS [04]: Retrieve User Permissions (0.00s)
        ℹ️ Note: No current user information available for permissions test
✅ PASS [05]: Retrieve User Roles (0.00s)
        ℹ️ Note: No current user information available for roles test
\n🏆 FINAL STATISTICS
----------------------------------------
📈 Success Rate: 5/5 (100.0%)
⏱️ Total Execution Time: 1.69s
👥 Users Discovered: 0
📋 User Fields Discovered: 0
👤 Current User Available: False
🔑 User Permissions Available: False
👑 User Roles Available: False
\n🏢 VAULT STATE
----------------------------------------
🔗 Connected to: https://vv-consulting-michael-mastermind.veevavault.com
🆔 Vault ID: 92425
👤 U